In [1]:
!pip install librosa
!pip install pesq
!pip install pystoi
!pip install natsort
import librosa
import numpy as np
from pystoi import stoi
import numpy as np
from scipy.signal import stft

import os
from natsort import natsorted
from pesq import pesq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.8/316.8 kB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 23.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 23.7 MB/s eta 0:00:00
  Created wheel for audioread: filename=audioread-3.0.0-py3-none-any.whl size=23706 sha256=48e1a6ca3a5e6f9eb5c51cf99ddcc8c58898779768512d99ca6b839d3f32a154
  Stored in directory: /root/.cache/pip/wheels/da/4b/39/c5f6c4ee93b43281dda4dab5ac5f2bdf9d11074d427493cd55
Successfully built audioread

[notice] A new release o

LSD

In [3]:
import librosa
import numpy as np
import os

# Define the paths of the subfolders
clean_folder = "/work/DNN/clean"
noise_folder = "/work/DNN/noisy"
enhance_folder = "/work/DNN/enhance"

# Initialize an empty list
# to store the LSD values
lsd_c_n=np.array([])
lsd_c_e=np.array([])


# read all audio from clean_folder and noise_folder
clean_files = os.listdir(clean_folder)
noise_files = os.listdir(noise_folder)
enhance_files=os.listdir(enhance_folder)

# sort the file names using natural sorting
clean_files = natsorted(clean_files)
noise_files = natsorted(noise_files)
sr=16000

# Loop through two audio file in clean_files and noise_files those flow natural sorting as an windows
#follow natural sorting while looping through the files in the folder
for clean_file,noise_file,enhance_file in zip(natsorted(clean_files),natsorted(noise_files),natsorted(enhance_files)):
    # Read the clean audio file
    clean_audio,sr=librosa.load(os.path.join(clean_folder,clean_file),sr=sr)
    # Read the noise audio file
    noise_audio,sr=librosa.load(os.path.join(noise_folder,noise_file),sr=sr)
    # Read the enhance audio file
    enhance_audio,sr=librosa.load(os.path.join(enhance_folder,enhance_file),sr=sr)
    # Calculate the LSD value
    if len(clean_audio) > len(noise_audio):
        clean_audio = clean_audio[:len(noise_audio)]
    else:
        noise_audio = noise_audio[:len(clean_audio)] 
 
    
    # Compute STFT of clean and noisy audio
    f, t, Zxx_noisy = stft(noise_audio, sr)
    f, t, Zxx_clean = stft(clean_audio, sr)
    #take log of the magnitude of the STFT
    log_Zxx_noisy = np.log(np.abs(Zxx_noisy))
    log_Zxx_clean = np.log(np.abs(Zxx_clean))
    # Compute mean squared error between log spectra
    mse = np.mean((log_Zxx_noisy - log_Zxx_clean) ** 2)

# Take square root to get LSD
    lsd = np.sqrt(mse)

# Print LSD value
    lsd_c_n = np.append(lsd_c_n, lsd)
    if len(clean_audio) > len(enhance_audio):
        clean_audio = clean_audio[:len(enhance_audio)]
    else:
        enhance_audio = enhance_audio[:len(clean_audio)]

# Compute STFT of enhance and clean audio
    f, t, Zxx_enhance = stft(enhance_audio, sr)
    f, t, Zxx_clean = stft(clean_audio, sr)
#take log of the magnitude of the STFT
    log_Zxx_enhance = np.log(np.abs(Zxx_enhance))
    log_Zxx_clean = np.log(np.abs(Zxx_clean))
# Compute mean squared error between log spectra
    mseenhance = np.mean((log_Zxx_enhance - log_Zxx_clean) ** 2)
    lsdenhance = np.sqrt(mseenhance)
    lsd_c_e = np.append(lsd_c_e, lsdenhance)

In [61]:
import pandas as pd
df = pd.DataFrame(clean_files, columns = ['File name']) #stoi_Noise_values   stoi_values
df['LSD C-N'] = lsd_c_n
df['LSD C-E']=lsd_c_e
df['LSD Distance Improvement']=df['LSD C-N']-df['LSD C-E']


In [65]:
DeepnoteChart(df, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#4c78a8","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"File name","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"LSD Distance Improvement","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

In [ ]:

# Initialize an empty list
# to store the LSD values
pesq_c_n=np.array([])
pesq_c_e=np.array([])


# read all audio from clean_folder and noise_folder
clean_files = os.listdir(clean_folder)
noise_files = os.listdir(noise_folder)
enhance_files=os.listdir(enhance_folder)

# sort the file names using natural sorting
clean_files = natsorted(clean_files)
noise_files = natsorted(noise_files)
sr=16000

# Loop through two audio file in clean_files and noise_files those flow natural sorting as an windows
#follow natural sorting while looping through the files in the folder
for clean_file,noise_file,enhance_file in zip(natsorted(clean_files),natsorted(noise_files),natsorted(enhance_files)):
    # Read the clean audio file
    clean_audio,sr=librosa.load(os.path.join(clean_folder,clean_file),sr=sr)
    # Read the noise audio file
    noise_audio,sr=librosa.load(os.path.join(noise_folder,noise_file),sr=sr)
    # Read the enhance audio file
    enhance_audio,sr=librosa.load(os.path.join(enhance_folder,enhance_file),sr=sr)
    # Calculate the STOI value
    if len(clean_audio) > len(noise_audio):
        clean_audio = clean_audio[:len(noise_audio)]
    else:
        noise_audio = noise_audio[:len(clean_audio)] 
 
    
    # Compute STFT of clean and noisy audio

In [12]:

# Initialize an empty list
# to store the LSD values
pesq_c_n=np.array([])
pesq_c_e=np.array([])


# read all audio from clean_folder and noise_folder
clean_files = os.listdir(clean_folder)
noise_files = os.listdir(noise_folder)
enhance_files=os.listdir(enhance_folder)

# sort the file names using natural sorting
clean_files = natsorted(clean_files)
noise_files = natsorted(noise_files)
sr=16000

# Loop through two audio file in clean_files and noise_files those flow natural sorting as an windows
#follow natural sorting while looping through the files in the folder
for clean_file,noise_file,enhance_file in zip(natsorted(clean_files),natsorted(noise_files),natsorted(enhance_files)):
    # Read the clean audio file
    clean_audio,sr=librosa.load(os.path.join(clean_folder,clean_file),sr=sr)
    # Read the noise audio file
    noise_audio,sr=librosa.load(os.path.join(noise_folder,noise_file),sr=sr)
    # Read the enhance audio file
    enhance_audio,sr=librosa.load(os.path.join(enhance_folder,enhance_file),sr=sr)
    # Calculate the STOI value
    if len(clean_audio) > len(noise_audio):
        clean_audio = clean_audio[:len(noise_audio)]
    else:
        noise_audio = noise_audio[:len(clean_audio)] 
 
    
    # Compute PESQ of clean and noisy audio
    # pesq between clean_audio,noise_audio
    pesq_socre_yc_yn = pesq(sr,clean_audio,noise_audio,'nb')
    pesq_c_n=np.append(pesq_c_n,pesq_socre_yc_yn)

    if len(clean_audio) > len(enhance_audio):
        clean_audio = clean_audio[:len(enhance_audio)]
    else:
        enhance_audio = enhance_audio[:len(clean_audio)]

    pesq_socre_yc_ye = pesq(sr,clean_audio,enhance_audio,'nb')
    # pesq between clean_audio,enhance_audio
    pesq_c_e=np.append(pesq_c_e,pesq_socre_yc_ye)

    

In [62]:
df["pesq-c-n"]=pesq_c_n
df["pesq-c-e"]=pesq_c_e
df["pesq_improvement"]=pesq_c_n-pesq_c_e

In [30]:
DeepnoteChart(df, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#4c78a8","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"File name","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"pesq_improvement","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

In [ ]:
import librosa
import numpy as np
import os
import soundfile as sf

import os
from natsort import natsorted

clean_folder = "/work/DNN/clean"
noise_folder = "/work/DNN/noisy"
enhance_folder="/work/DNN/enhance"

# read all audio from clean_folder and noise_folder
clean_files = os.listdir(clean_folder)
noise_files = os.listdir(noise_folder)
enhance_files=os.listdir(enhance_folder)
# sort the file names using natural sorting
clean_files = natsorted(clean_files)
noise_files = natsorted(noise_files)
enhance_files=natsorted(enhance_files)
print(clean_files)
print(noise_files)

# Initialize an empty list to store the LSD values
stoi_c_e=np.array([])
stoi_c_n=np.array([])
# Loop through two audio file in clean_files and noise_files those flow natural sorting as an windows
#follow natural sorting while looping through the files in the folder
for clean_file,noise_file,enhance_file in zip(natsorted(clean_files),natsorted(noise_files),natsorted(enhance_files)):
     # Read the clean audio file
    clean_audio,sr=librosa.load(os.path.join(clean_folder,clean_file),sr=sr)
    # Read the noise audio file
    noise_audio,sr=librosa.load(os.path.join(noise_folder,noise_file),sr=sr)
    # Read the enhance audio file
    enhance_audio,sr=librosa.load(os.path.join(enhance_folder,enhance_file),sr=sr)
    if len(clean_audio) > len(noise_audio):
        clean_audio = clean_audio[:len(noise_audio)]
    else:
        noise_audio = noise_audio[:len(clean_audio)] 
    stoi_value=stoi(clean_audio,noise_audio,fs,extended=False)
    # Append the stoi_value to the list
    stoi_c_n=np.append(stoi_c_n,stoi_value)
    if len(clean_audio) > len(enhance_audio):
        clean_audio = clean_audio[:len(enhance_audio)]
    else:
        enhance_audio = enhance_audio[:len(clean_audio)] 
    stoi_value1=stoi(clean_audio,enhance_audio,fs,extended=False)
    # Append the stoi_value to the list
    stoi_c_e=np.append(stoi_c_e,stoi_value1)

In [63]:
df["stoi-c-e"]=stoi_c_e
df["stoi-c-n"]=stoi_c_n
df["stoi improvement"]=stoi_c_e-stoi_c_n

In [59]:
DeepnoteChart(df, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#4c78a8","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"File name","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"stoi improvement","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9ff8dd5a-42ba-46e9-ae3d-27d5566d3c4b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>